In [11]:
import torch
from model import LayoutLMT5
from metrics import compute_f1, compute_exact

In [2]:
model = LayoutLMT5.load_from_checkpoint("models/baseline/baseline-epoch=9.ckpt", strict=False).eval().cuda()

Initializing T5...


Some weights of the model checkpoint at t5-base were not used when initializing T5ForConditionalGeneration: ['decoder.block.0.layer.1.EncDecAttention.relative_attention_bias.weight']
- This IS expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [27]:
model.hparams.bs = 10
val_dataloader = model.val_dataloader()

val DocVQA folder data/raw/val tokenizer PreTrainedTokenizer(name_or_path='t5-base', vocab_size=32100, model_max_len=512, is_fast=False, padding_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>', '<extra_i

In [28]:
batch = next(iter(val_dataloader))
batch

{'document': ['NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA', 'NA'],
 'input_ids': tensor([[ 822,   10,  363,  ...,    0,    0,    0],
         [ 822,   10,  363,  ...,    0,    0,    0],
         [ 822,   10,  363,  ...,    0,    0,    0],
         ...,
         [ 822,   10,  363,  ...,    0,    0,    0],
         [ 822,   10,  363,  ...,    0,    0,    0],
         [ 822,   10, 2150,  ...,    0,    0,    0]]),
 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'input_text': ['question: What is the ‘actual’ value per 1000, during the year 1975? inpu

In [29]:
cuda_batch = {}
for k, v in batch.items():
    if k in ["document", "target_text", "input_text"]:
        cuda_batch[k] = v
    else:
        cuda_batch[k] = v.cuda()  
batch = cuda_batch

In [30]:
with torch.no_grad():
    output = model(batch)

In [31]:
tokenizer = model.tokenizer
tokenizer

PreTrainedTokenizer(name_or_path='t5-base', vocab_size=32100, model_max_len=512, is_fast=False, padding_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>', '<extra_id_43>', '<extra_id_44>', '<extra_id_45>',

In [41]:
output_text = [tokenizer.decode(out, skip_special_tokens=True) for out in output]
tgt_text = batch["target_text"]
for i in range(10):
    print(f"prediction: {output_text[i]}\ntarget: {tgt_text[i]}\nF1: {compute_f1(output_text[i], tgt_text[i])}, Exact: {compute_exact(output_text[i], tgt_text[i])}\n")

prediction: 8.28
target: 0.28
F1: 0, Exact: 0

prediction: UNIVERSITY OF CALIFORNIA, SAN DIEGO
target: university of california
F1: 0.5, Exact: 0

prediction: ITC Limited
target: itc limited
F1: 1.0, Exact: 1

prediction: UNIVERSITY OF CALIFORNIA, SAN DIEGO
target: san diego
F1: 0.5714285714285715, Exact: 0

prediction: Dr. Ms.
target: Paul
F1: 0, Exact: 0

prediction: 1128 SIXTEENTH ST., WASHINGTON, D.C. 20036
target: 1128 SIXTEENTH ST., N. W., WASHINGTON, D. C. 20036
F1: 0.6666666666666667, Exact: 0

prediction: An Asset for the Nation
target: aashirvaad
F1: 0, Exact: 0

prediction: THE ROBERT A. WELCH FOUNDATION
target: The Robert A. Welch Foundation
F1: 1.0, Exact: 1

prediction: 11:14 to 5:00 p.m.
target: 11:14 to 11:39 a.m.
F1: 0.5, Exact: 0

prediction: $ 975.00
target: $975.00
F1: 0, Exact: 0

